In [2]:
import pandas as pd
import pathlib as Path
from transformers import AutoTokenizer

train_path = Path.Path("../data/raw/train.csv")
test_path = Path.Path("../data/raw/test.csv")

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

example_text = train_df['text'].iloc[0]
print(f"Example text: {example_text}")
encoded_example = tokenizer(example_text, return_tensors='pt')
print(f"Encoded example: {encoded_example}")

Example text: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and fa

In [3]:
def tokenize_text(text, tokenizer, max_length=128):
    return tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt')

train_encodings = tokenize_text(train_df['text'].tolist(), tokenizer)
test_encodings = tokenize_text(test_df['text'].tolist(), tokenizer)

In [ ]:
import torch
from torch.utils.data import Dataset

class IMDbDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = IMDbDataset(train_encodings, train_df["label"].tolist())
test_dataset = IMDbDataset(test_encodings, test_df["label"].tolist())

In [9]:
from torch.utils.data import DataLoader
batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)